In [1]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
from sklearn.model_selection import cross_val_score


In [2]:
app = Flask(__name__)

In [3]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')

In [4]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
data_fake["class"]= 0
data_true["class"]= 1


In [6]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [7]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
     data_fake.drop([i] , axis = 0, inplace =True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
     data_true.drop([i] , axis = 0, inplace =True)


In [8]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [9]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

C:\Users\Asus\AppData\Local\Temp\ipykernel_17388\277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\Asus\AppData\Local\Temp\ipykernel_17388\277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [10]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [11]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [12]:
data_merge = pd.concat([data_fake,data_true],axis = 0)
data_merge.sample(10)

,title,text,subject,date,class
50,Sources Confirm Robert Mueller’s Office Inter...,"Jared Kushner, senior White House adviser and ...",News,"November 30, 2017",0
6484,U.S. Congress certifies Trump's Electoral Coll...,WASHINGTON (Reuters) - The U.S. Congress on Fr...,politicsNews,"January 6, 2017",1
7244,Canadian Company Offers Asylum To Americans L...,A Canadian tech company is capitalizing on Don...,News,"March 28, 2016",0
4286,Lifelong Republican Arnold Schwarzenegger TER...,I ll be back is not what Arnold Schwarzenegg...,News,"October 9, 2016",0
1407,Comey Hates Trump So Much He Once Tried To Hi...,We all know the feeling of dread that takes ov...,News,"May 19, 2017",0
18426,French union rank and file urge their bosses t...,PARIS (Reuters) - Rank and file members of Fra...,worldnews,"October 3, 2017",1
2707,"South Park Creators GIVE UP, Say Donald Trump...",The 20th season of South Park was largely cent...,News,"February 3, 2017",0
15094,"China faces historic corruption battle, ruling...",SHANGHAI (Reuters) - China must win its battle...,worldnews,"November 11, 2017",1
16585,CLIMATE CHANGE REFUGEES? FEDS GIVE $1 BILLION ...,Now it beings the government has changed who i...,Government News,"May 30, 2016",0
14281,"“We choose the nominee, not the people” – GOP ...",The media has created the perception that the...,politics,"Mar 16, 2016",0


In [13]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [14]:
data = data_merge.drop(['title','subject','date'],axis = 1)  

In [15]:
data.isnull().sum()


text     0
class    0
dtype: int64

In [16]:
data = data.sample(frac = 1)

In [17]:
data.head()

,text,class
12109,,0
14963,Michelle Obama received $70 million dollars fr...,0
8835,"Let s face it, Republicans hate Neil Degrasse ...",0
9932,Leftist activist and mediocre filmmaker Michae...,0
6346,LONDON (Reuters) - British Prime Minister Ther...,1


In [18]:
data.reset_index(inplace =True)
data.drop(['index'],axis = 1, inplace =True)

In [19]:
data.columns

Index(['text', 'class'], dtype='object')

In [20]:
data.head()

,text,class
0,,0
1,Michelle Obama received $70 million dollars fr...,0
2,"Let s face it, Republicans hate Neil Degrasse ...",0
3,Leftist activist and mediocre filmmaker Michae...,0
4,LONDON (Reuters) - British Prime Minister Ther...,1


In [21]:
import string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text









In [22]:
data['text'] = data['text'].apply(wordopt)

In [23]:
x= data['text']
y = data['class']

In [24]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)


In [26]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
# Perform 5-fold cross-validation and print the scores
scores = cross_val_score(LR, xv_train, y_train, cv=5)  # 5-fold cross-validation
print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", scores.mean())  # Mean accuracy of the cross-validation

LR.fit(xv_train,y_train)

Cross-Validation Scores: [0.98603684 0.98291741 0.98366013 0.98425197 0.98662903]
Mean Accuracy: 0.9846990754901477


LogisticRegression()

In [27]:
pred_lr = LR.predict(xv_test)

In [28]:
LR.score(xv_test, y_test)

0.9875222816399287

In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5905
           1       0.98      0.99      0.99      5315

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [30]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [31]:
pred_dt =DT.predict(xv_test)

In [32]:
DT.score(xv_test, y_test)

0.9957219251336898

In [33]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5905
           1       1.00      0.99      1.00      5315

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [34]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

KeyboardInterrupt: 

In [ ]:
pred_gb = GB.predict(xv_test)

In [ ]:
GB.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_gb))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

In [ ]:
pred_rf = RF.predict(xv_test)

In [ ]:
RF.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_rf))

In [ ]:
import pandas as pd
  # You should import the vectorization module


def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"

# Define the manual_testing function
def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)  # Assuming wordopt is defined elsewhere
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)  # Assuming vectorization is defined and working correctly
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    pred_Xgb = 

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(
        output_label(pred_LR[0]),
        output_label(pred_DT[0]),
        output_label(pred_GB[0]),
        output_label(pred_RF[0])
    ))



In [ ]:
# from flask import Flask

# @app.route('/predict', methods=['POST'])
# def predict():
#     if request.method == 'POST':
#         news_text = request.form['input_text']
#         if not news_text:
#             return render_template('model_page.html', prediction="Please enter news text.")
#         else:
           
#              manual_testing(news_text)
           
#         return render_template('model_page.html', manual_testing=f"The text is classified as:")
        
# if __name__ == '__main__':
#      app.run( debug=True)

In [35]:
import xgboost as xgb

# Assuming xv_train and y_train are your training data (features and target variable)

# Create an XGBoost classifier object
model = xgb.XGBClassifier(objective='binary:logistic', random_state=0)  # Adjust 'objective' for multi-class

# Fit the model to your training data
model.fit(xv_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...)

In [36]:
pred_Xgb = model.predict(xv_test)

In [37]:
model.score(xv_test, y_test)

0.9978609625668449

In [39]:
print(classification_report(y_test, pred_Xgb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5905
           1       1.00      1.00      1.00      5315

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



# New Section